In [21]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [29]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("World_Weather_Analysis/WeatherPy_DB.csv.ipynb")
city_data_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'World_Weather_Analysis/WeatherPy_DB.csv.ipynb'

In [3]:
#Get the Data types
city_data_df.dtypes


City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Get locations 
locations = city_data_df[["Lat","Lng"]]

# Get temperatures of locations 
max_temp = city_data_df["Max Temp"]

# Create figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)

# Create heatmap layer
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp],
                                 dissipating=False, max_intensity=300, point_radius=4)

# Combine heatmap to figure
fig.add_layer(heat_layer)

# Show figure
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent Cloudiness

locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
locations = city_data_df[["Lat", "Lng"]]
Wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=Wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [11]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Namibe,AO,2022-09-30 05:46:28,-15.1961,12.1522,68.41,88,35,2.82
2,2,Rikitea,PF,2022-09-30 05:46:28,-23.1203,-134.9692,72.54,69,5,4.45
4,4,Ribeira Grande,PT,2022-09-30 05:46:28,38.5167,-28.7000,66.60,72,75,6.91
7,7,Galle,LK,2022-09-30 05:46:29,6.0367,80.2170,81.88,78,73,16.37
8,8,East London,ZA,2022-09-30 05:46:30,-33.0153,27.9116,67.14,82,0,6.91
11,11,Orange Walk,BZ,2022-09-30 05:46:31,17.8333,-88.8333,69.64,82,100,3.85
14,14,Busselton,AU,2022-09-30 05:46:32,-33.6500,115.3333,73.54,43,0,4.25
16,16,Kruisfontein,ZA,2022-09-30 05:46:32,-34.0033,24.7314,63.86,84,100,3.87
18,18,Grand Gaube,MU,2022-09-30 05:46:33,-20.0064,57.6608,75.38,63,17,8.99
19,19,Kavkazskaya,RU,2022-09-30 05:46:34,45.4450,40.6728,68.56,60,32,5.08


In [12]:
preferred_cities_df.count()

City_ID       337
City          337
Country       336
Date          337
Lat           337
Lng           337
Max Temp      337
Humidity      337
Cloudiness    337
Wind Speed    337
dtype: int64

In [13]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)


,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Namibe,AO,68.41,-15.1961,12.1522,
2,Rikitea,PF,72.54,-23.1203,-134.9692,
4,Ribeira Grande,PT,66.60,38.5167,-28.7000,
7,Galle,LK,81.88,6.0367,80.2170,
8,East London,ZA,67.14,-33.0153,27.9116,
11,Orange Walk,BZ,69.64,17.8333,-88.8333,
14,Busselton,AU,73.54,-33.6500,115.3333,
16,Kruisfontein,ZA,63.86,-34.0033,24.7314,
18,Grand Gaube,MU,75.38,-20.0064,57.6608,
19,Kavkazskaya,RU,68.56,45.4450,40.6728,


In [14]:
# Set parameters to search for a hotel.
params = {
    "radius": 3000,
    "type": "lodging",
    "key": g_key
}

In [15]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")


ReadTimeout: HTTPSConnectionPool(host='maps.googleapis.com', port=443): Read timed out. (read timeout=None)

In [ ]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

AttributeError: module 'collections' has no attribute 'Iterable'

In [ ]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [ ]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


In [ ]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

AttributeError: module 'collections' has no attribute 'Iterable'